In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_colwidth=999
import glob
train = pd.DataFrame({'path' : glob.glob('/kaggle/input/plant-seedlings-classification/train/*/*')})
train

In [ ]:
train['label'] = train['path'].apply(lambda x: x.split('/')[-2])
train

In [ ]:
train['label'].value_counts()
# 다중분류 문제 --> 정답값이 한쪽으로 쏠려 있으므로 모델 학습시 metrics 등의 옵션을 다르게 줘야할수도!

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid = train_test_split(train,test_size=0.2,random_state=42,
                                  stratify=train['label'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator(horizontal_flip=True)
# horizontal_flip : 원본사진중에서 랜덤하게 50% 골라 좌우반전되는 사진 추가시켜 학습
# --> 원본을 모두 학습못할수 있기 때문에 epochs를 2이상으로 늘려서 보정!
train_generator = idg.flow_from_dataframe(X_train,x_col='path',y_col='label',
                                         target_size=(200,200),batch_size=64)
valid_generator = idg.flow_from_dataframe(X_valid,x_col='path',y_col='label',
                                         target_size=(200,200),batch_size=64)
# batch_size가 적을수록 좋은이유? -> batch_size가 적을수록 훨씬 디테일하게 좋은부분/안좋은부분 가릴수 있음

In [ ]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')

es = EarlyStopping(patience=3,restore_best_weights=True,verbose=1)
rl = ReduceLROnPlateau(patience=2)
# ReduceLROnPlateau : 모델이 학습하다 막히면(train loss는 낮아지지만 val_loss가 안좋아지는 경우)
#                    learning_rate를 낮춰주겠다(더 자세히 학습)
#                   --> patience는 es보다 적게 줘야함
efn = EfficientNetB0(include_top=False,pooling='avg')
model = Sequential()
model.add(efn)
model.add(Dense(12,activation='softmax'))
model.compile(metrics=['acc'],loss='categorical_crossentropy',optimizer='adam')
model.fit(train_generator,validation_data=valid_generator,epochs=100,callbacks=[es,rl])

In [ ]:
test = pd.DataFrame({'path':glob.glob('/kaggle/input/plant-seedlings-classification/test/*')})
test

In [ ]:
test_generator = idg.flow_from_dataframe(test,x_col='path',y_col=None,
                                        class_mode=None,shuffle=False,target_size=(200,200),batch_size=64)

In [ ]:
result = model.predict(test_generator,verbose=1)
result[0]  # 총 12개 클래스에 대한 모델의 확률값

In [ ]:
train_generator.class_indices
# train_generator의 label 값

In [ ]:
sub = pd.read_csv('/kaggle/input/plant-seedlings-classification/sample_submission.csv')
sub
# 정답 확률이 아닌 species=sugar beet와 같은 가장 확률 높은 정답값을 답으로 제출해야함!

In [ ]:
result.argmax(1)
# argmax() : 가장 확률값이 높은 인덱스를 가져오는 함수(axis=0 --> 열방향 최대값, axis=1 --> 행방향 최대값)

In [ ]:
class_index = list(train_generator.class_indices)
class_index

In [ ]:
result_classes = []

for i in result.argmax(1):
    label = class_index[i]
    result_classes.append(label)
result_classes[:5]

In [ ]:
sub['species'] = result_classes
sub['file'] = test['path'].apply(lambda x: x.split('/')[-1])
sub.to_csv('result.csv',index=False)